In [1]:
import zipfile
import tarfile

from os import listdir
from os.path import isfile, join
from pathlib import Path

from abc import ABC, abstractmethod

import random
from tqdm import tqdm

In [2]:
def instanceAllObjects(className, bytesArray, count, size):
    instanceList = []
    startByte = 0
    constructor = globals()[className]
    for o in range(0, count):
        instance = constructor(bytesArray[startByte:startByte + size])
        startByte += size
        instanceList.append(instance)
    return instanceList

In [3]:
def addZeros(bytesArray, finalSize):
    if(len(bytesArray) < finalSize):
        missingBytes = finalSize - len(bytesArray)
        zero = 0
        zeroByte = zero.to_bytes(1, 'little')
        bytesArray = bytesArray + (zeroByte * missingBytes)
    return bytesArray  

In [4]:
def fromListToBytes(objects, finalSize):
    objectsBytes =  bytearray()
    if len(objects) > 0:
        objectsBytesLists = list(map(lambda o: o.toBytes(), objects))
        objectsBytes = objectsBytesLists[0]
        for ob in objectsBytesLists[1:]:
            objectsBytes += ob
    return addZeros(objectsBytes, finalSize)    
    

In [5]:
class ObjectBytes(ABC):
    @abstractmethod
    def toBytes(self):
        pass

In [6]:
class GameObject(ObjectBytes):
    def __init__(self, objectBytes):
        self.xPosition = int.from_bytes(objectBytes[:4],"little", signed = False)
        self.yPosition = int.from_bytes(objectBytes[4:8],"little", signed = False)
        self.unUsed = int.from_bytes(objectBytes[8:10],"little", signed = False)
        self.objectWidth = objectBytes[10]
        self.objectHeight = objectBytes[11]
        self.objectFlags = int.from_bytes(objectBytes[12:16],"little", signed = False)
        self.childObjectFlags = int.from_bytes(objectBytes[16:20],"little", signed = False)
        self.extendedData = int.from_bytes(objectBytes[20:24],"little", signed = False)
        self.objectType = int.from_bytes(objectBytes[24:26],"little", signed = False)
        self.childObjectType = int.from_bytes(objectBytes[26:28],"little", signed = False)
        self.linkID = int.from_bytes(objectBytes[28:30],"little", signed = False)
        self.soundEffect = int.from_bytes(objectBytes[30:32],"little", signed = False)
    
    def toBytes(self):
        xBytes = self.xPosition.to_bytes(4, 'little')
        yBytes = self.yPosition.to_bytes(4, 'little')
        unUsedBytes = self.unUsed.to_bytes(2, 'little')
        width = self.objectWidth.to_bytes(1, 'little')
        height = self.objectHeight.to_bytes(1, 'little')
        flags = self.objectFlags.to_bytes(4, 'little')
        childObjectFlags = self.childObjectFlags.to_bytes(4, 'little')
        extendedData = self.extendedData.to_bytes(4, 'little')
        objectType = self.objectType.to_bytes(2, 'little')
        childObjectType = self.childObjectType.to_bytes(2, 'little')
        linkID = self.linkID.to_bytes(2, 'little')
        soundEffect = self.soundEffect.to_bytes(2, 'little')
        return xBytes + yBytes + unUsedBytes + width + height + flags + childObjectFlags + extendedData + objectType + childObjectType + linkID + soundEffect

In [7]:
class FreestandingSoundEffect(ObjectBytes):
    def __init__(self, soundBytes):
        self.effectType = soundBytes[0]
        self.xPosition = soundBytes[1]
        self.yPosition = soundBytes[2]
        self.padding = soundBytes[3]
        
    def toBytes(self):
        effectType = self.effectType.to_bytes(1, 'little')
        xPosition = self.xPosition.to_bytes(1, 'little')
        yPosition = self.yPosition.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        return effectType + xPosition + yPosition + padding

In [8]:
class SnakeNode(ObjectBytes):
    def __init__(self, snakeNodeBytes):
        self.index = int.from_bytes(snakeNodeBytes[:2],"little", signed = False) #parte da zero
        self.mistery = int.from_bytes(snakeNodeBytes[2:4],"little", signed = False)
        self.zerocento = int.from_bytes(snakeNodeBytes[4:6],"little", signed = False)
        self.unused = int.from_bytes(snakeNodeBytes[6:8],"little", signed = False) 
        
    def toBytes(self):
        index = self.index.to_bytes(2, 'little')
        mistery = self.mistery.to_bytes(2, 'little')
        zerocento = self.zerocento.to_bytes(2, 'little')
        unused = self.unused.to_bytes(2, 'little')
        return index + mistery + zerocento + unused

In [9]:
class SnakeBlockTrack(ObjectBytes):
    def __init__(self, snakeBytes):
        self.snakeID = snakeBytes[0] #sarà 0 1 2 3 4?
        self.nodeCount = snakeBytes[1]
        self.always1 = snakeBytes[2]
        self.padding = snakeBytes[3]
        self.snakeNodes = instanceAllObjects('SnakeNode', snakeBytes[4:], self.nodeCount, 8)
        
    def toBytes(self):
        snakeID = self.snakeID.to_bytes(1, 'little')
        nodeCount = self.nodeCount.to_bytes(1, 'little')
        always1 = self.always1.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        snakeNodes = fromListToBytes(self.snakeNodes, 960)
        return snakeID + nodeCount + always1 + padding + snakeNodes  

In [10]:
class ClearPipeNode(ObjectBytes):
    def __init__(self, clearPipeNodesBytes):
        self.unknown011 = clearPipeNodesBytes[0]
        self.index = clearPipeNodesBytes[1]
        self.unKnowCoordinates = clearPipeNodesBytes[2]
        self.unKnowCoordinates2 = clearPipeNodesBytes[3]
        self.always2 = clearPipeNodesBytes[4]
        self.unknown = clearPipeNodesBytes[5] 
        self.always1 = clearPipeNodesBytes[6]
        self.unknown0123 = clearPipeNodesBytes[7]
        
    def toBytes(self):
        unknown011 = self.unknown011.to_bytes(1, 'little')
        index = self.index.to_bytes(1, 'little')
        unKnowCoordinates = self.unKnowCoordinates.to_bytes(1, 'little')
        unKnowCoordinates2 = self.unKnowCoordinates2.to_bytes(1, 'little')
        always2 = self.always2.to_bytes(1, 'little')
        unknown = self.unknown.to_bytes(1, 'little')
        always1 = self.always1.to_bytes(1, 'little')
        unknown0123 = self.unknown0123.to_bytes(1, 'little')
        return unknown011 + index + unKnowCoordinates + unKnowCoordinates2 + always2 + unknown + always1 + unknown0123

In [11]:
class ClearPipe(ObjectBytes):
    def __init__(self, clearPipeBytes):
        self.id = clearPipeBytes[0]
        self.nodeCount = clearPipeBytes[1]
        self.always1 = clearPipeBytes[2]
        self.padding = clearPipeBytes[3]
        self.clearPipeNodes = instanceAllObjects('ClearPipeNode', clearPipeBytes[4:], self.nodeCount, 8)
        
    def toBytes(self):
        pipeId = self.id.to_bytes(1, 'little')
        nodeCount = self.nodeCount.to_bytes(1, 'little')
        always1 = self.always1.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        clearPipeNodes = fromListToBytes(self.clearPipeNodes, 288)
        return pipeId + nodeCount + always1 + padding + clearPipeNodes  

In [12]:
class PiranhaCreeperNode(ObjectBytes):
    def __init__(self, piranhaCreeperNodeBytes):
        self.always1 = piranhaCreeperNodeBytes[0] 
        self.valuesSeen = piranhaCreeperNodeBytes[1]
        self.always0_1 = piranhaCreeperNodeBytes[2]
        self.always0_2 = piranhaCreeperNodeBytes[3]
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        valuesSeen = self.valuesSeen.to_bytes(1, 'little')
        always0_1 = self.always0_1.to_bytes(1, 'little')
        always0_2 = self.always0_2.to_bytes(1, 'little')
        return always1 + valuesSeen + always0_1 + always0_2

In [13]:
class PiranhaCreeperTrack(ObjectBytes):
    def __init__(self, piranhaCreeperTrackBytes):
        self.always1 = piranhaCreeperTrackBytes[0] 
        self.idTrack = piranhaCreeperTrackBytes[1]
        self.nodeCount = piranhaCreeperTrackBytes[2]
        self.padding = piranhaCreeperTrackBytes[3]
        self.piranhaCreeperNodes = instanceAllObjects('PiranhaCreeperNode', piranhaCreeperTrackBytes[4:], self.nodeCount, 4)
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        idTrack = self.idTrack.to_bytes(1, 'little')
        nodeCount = self.nodeCount.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        piranhaCreeperNodes = fromListToBytes(self.piranhaCreeperNodes, 80)
        return always1 + idTrack + nodeCount + padding + piranhaCreeperNodes  

In [14]:
class ExpandingBlockNode(ObjectBytes):
    def __init__(self, expandingBlockNodeBytes):
        self.always1 = expandingBlockNodeBytes[0] 
        self.valuesSeen = expandingBlockNodeBytes[1]
        self.always0_1 = expandingBlockNodeBytes[2]
        self.always0_2 = expandingBlockNodeBytes[3]  
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        valuesSeen = self.valuesSeen.to_bytes(1, 'little')
        always0_1 = self.always0_1.to_bytes(1, 'little')
        always0_2 = self.always0_2.to_bytes(1, 'little')
        return always1 + valuesSeen + always0_1 + always0_2   

In [15]:
class ExpandingBlockTrack(ObjectBytes):
    def __init__(self, expandingBlockTrackBytes):
        self.always1 = expandingBlockTrackBytes[0] 
        self.idTrack = expandingBlockTrackBytes[1]
        self.nodeCount = expandingBlockTrackBytes[2]
        self.padding = expandingBlockTrackBytes[3]    
        self.expandingBlockNodes = instanceAllObjects('ExpandingBlockNode', expandingBlockTrackBytes[4:], self.nodeCount, 4)
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        idTrack = self.idTrack.to_bytes(1, 'little')
        nodeCount = self.nodeCount.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        expandingBlockNodes = fromListToBytes(self.expandingBlockNodes, 40)
        return always1 + idTrack + nodeCount + padding + expandingBlockNodes      

In [16]:
class TrackBlockNode(ObjectBytes):
    def __init__(self, trackBlockNodeBytes):
        self.always1 = trackBlockNodeBytes[0]
        self.valuesSeen = trackBlockNodeBytes[1]
        self.always0_1 = trackBlockNodeBytes[2]
        self.always0_2 = trackBlockNodeBytes[3]
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        valuesSeen = self.valuesSeen.to_bytes(1, 'little')
        always0_1 = self.always0_1.to_bytes(1, 'little')
        always0_2 = self.always0_2.to_bytes(1, 'little')
        return always1 + valuesSeen + always0_1 + always0_2 

In [17]:
class TrackBlockTrack(ObjectBytes):
    def __init__(self, trackBlockTrackBytes):
        self.always1 = trackBlockTrackBytes[0] 
        self.idTrackBlock = trackBlockTrackBytes[1]
        self.nodeCount = trackBlockTrackBytes[2]
        self.unused = trackBlockTrackBytes[3]  
        self.trackBlockNodes = instanceAllObjects('TrackBlockNode', trackBlockTrackBytes[4:], self.nodeCount, 4)
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        idTrackBlock = self.idTrackBlock.to_bytes(1, 'little')
        nodeCount = self.nodeCount.to_bytes(1, 'little')
        unused = self.unused.to_bytes(1, 'little')
        trackBlockNodes = fromListToBytes(self.trackBlockNodes, 40)
        return always1 + idTrackBlock + nodeCount + unused + trackBlockNodes  

In [18]:
class Tile(ObjectBytes):
    def __init__(self, tileBytes):
        self.xPosition = tileBytes[0]
        self.yPosition = tileBytes[1]
        self.tileID = int.from_bytes(tileBytes[2:4],"little", signed = False) 
        
    def toBytes(self):
        xPosition = self.xPosition.to_bytes(1, 'little')
        yPosition = self.yPosition.to_bytes(1, 'little')
        tileID = self.tileID.to_bytes(2, 'little')
        return xPosition + yPosition + tileID      

In [19]:
class Rail(ObjectBytes):
    def __init__(self, railBytes):
        self.alwaysZero = int.from_bytes(railBytes[:2],"little", signed = False) 
        self.alwaysZeroOne = railBytes[2]
        self.xPosition = railBytes[3]
        self.yPosition = railBytes[4]
        self.unknown014 = railBytes[5]
        self.index = int.from_bytes(railBytes[6:8],"little", signed = False) 
        self.unknown2 = int.from_bytes(railBytes[8:10],"little", signed = False) 
        self.unknown3 = int.from_bytes(railBytes[10:12],"little", signed = False) 
    def toBytes(self):
        alwaysZero = self.alwaysZero.to_bytes(2, 'little')
        alwaysZeroOne = self.alwaysZeroOne.to_bytes(1, 'little')
        xPosition = self.xPosition.to_bytes(1, 'little')
        yPosition = self.yPosition.to_bytes(1, 'little')
        unknown014 = self.unknown014.to_bytes(1, 'little')
        index = self.index.to_bytes(2, 'little')
        unknown2 = self.unknown2.to_bytes(2, 'little')
        unknown3 = self.unknown3.to_bytes(2, 'little')
        return alwaysZero + alwaysZeroOne + xPosition + yPosition + unknown014 + index + unknown2 + unknown3

In [20]:
class Icicle(ObjectBytes):
    def __init__(self, icicleBytes):
        self.xPosition = icicleBytes[0]
        self.yPosition = icicleBytes[1]
        self.zeroOne = icicleBytes[2]
        self.padding = icicleBytes[3]
    def toBytes(self):
        xPosition = self.xPosition.to_bytes(1, 'little')
        yPosition = self.yPosition.to_bytes(1, 'little')
        zeroOne = self.zeroOne.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        return xPosition + yPosition + zeroOne + padding

In [21]:
class LevelArea(ObjectBytes):
    def __init__(self, levelBytes):
        self.courseTheme = levelBytes[0]
        self.autoScrollType = levelBytes[1]
        self.screenBoundaryFlags = levelBytes[2]
        self.levelHorientation = levelBytes[3]
        self.lavaWaterHeight = levelBytes[4]
        self.lavaWaterMode = levelBytes[5]
        self.lavaWaterSpeed = levelBytes[6]
        self.minimunLavaWaterHeight = levelBytes[7]
        self.rightBoundary = int.from_bytes(levelBytes[8:12],"little", signed = False)
        self.topBoundary = int.from_bytes(levelBytes[12:16],"little", signed = False)
        self.leftBoundary = int.from_bytes(levelBytes[16:20],"little", signed = False)
        self.bottomBoundary = int.from_bytes(levelBytes[20:24],"little", signed = False)
        self.areaFlags = int.from_bytes(levelBytes[24:28],"little", signed = False)
        self.objectCount = int.from_bytes(levelBytes[28:32],"little", signed = False)
        self.freeStandingSoundEffectCount = int.from_bytes(levelBytes[32:36],"little", signed = False)
        self.snakeBlockCount = int.from_bytes(levelBytes[36:40],"little", signed = False)
        self.clearPipeCount = int.from_bytes(levelBytes[40:44],"little", signed = False)
        self.piranhaCreeperCount = int.from_bytes(levelBytes[44:48],"little", signed = False)
        self.expandingBlockCount = int.from_bytes(levelBytes[48:52],"little", signed = False)
        self.trackBlockCount = int.from_bytes(levelBytes[52:56],"little", signed = False)
        self.alwaysZero = int.from_bytes(levelBytes[56:60],"little", signed = False)
        self.tileCount = int.from_bytes(levelBytes[60:64],"little", signed = False)
        self.railCount = int.from_bytes(levelBytes[64:68],"little", signed = False)
        self.icicleCount = int.from_bytes(levelBytes[68:72],"little", signed = False)
        
        
        
        objectsBytes = levelBytes[72:83272]
        self.objects = instanceAllObjects('GameObject', objectsBytes, self.objectCount, 32)
        
        
        soundEffectsBytes = levelBytes[83272:84472]
        self.soundEffects = instanceAllObjects('FreestandingSoundEffect', soundEffectsBytes, self.freeStandingSoundEffectCount, 4)
        
        
        snakesBytes = levelBytes[84472:89292]
        self.snakes = instanceAllObjects('SnakeBlockTrack', snakesBytes, self.snakeBlockCount, 964)
        
        
        clearPipeBytes = levelBytes[89292:147692]
        self.clearPipes = instanceAllObjects('ClearPipe', clearPipeBytes, self.clearPipeCount, 292)
        
        creeperBytes = levelBytes[147692:148532]
        self.creepers = instanceAllObjects('PiranhaCreeperTrack', creeperBytes, self.piranhaCreeperCount, 84)
        
        expandingBlockBytes = levelBytes[148532:148972]
        self.expandingBlocks = instanceAllObjects('ExpandingBlockTrack', expandingBlockBytes, self.expandingBlockCount, 44)
        
        trackBlockBytes = levelBytes[148972:149412]
        self.trackBlocks = instanceAllObjects('TrackBlockTrack', trackBlockBytes, self.trackBlockCount, 44)
        
        tilesBytes = levelBytes[149412:165412]
        self.tiles = instanceAllObjects('Tile', tilesBytes, self.tileCount, 4)
        
        railBytes = levelBytes[165412:183412]
        self.rails = instanceAllObjects('Rail', railBytes, self.railCount, 12)
        
        iCicleBytes = levelBytes[183412:184612]
        self.icicles = instanceAllObjects('Icicle', iCicleBytes, self.icicleCount, 4)
        
    def toBytes(self):
        courseTheme = self.courseTheme.to_bytes(1, 'little')
        autoScrollType = self.autoScrollType.to_bytes(1, 'little')
        screenBoundaryFlags = self.screenBoundaryFlags.to_bytes(1, 'little')
        levelHorientation = self.levelHorientation.to_bytes(1, 'little')
        lavaWaterHeight = self.lavaWaterHeight.to_bytes(1, 'little')
        lavaWaterMode = self.lavaWaterMode.to_bytes(1, 'little')
        lavaWaterSpeed = self.lavaWaterSpeed.to_bytes(1, 'little')
        minimunLavaWaterHeight = self.minimunLavaWaterHeight.to_bytes(1, 'little')
        rightBoundary = self.rightBoundary.to_bytes(4, 'little')
        topBoundary = self.topBoundary.to_bytes(4, 'little')
        leftBoundary = self.leftBoundary.to_bytes(4, 'little')
        rightBoundary = self.rightBoundary.to_bytes(4, 'little')
        bottomBoundary = self.bottomBoundary.to_bytes(4, 'little')
        areaFlags = self.areaFlags.to_bytes(4, 'little')
        objectCount = self.objectCount.to_bytes(4, 'little')
        freeStandingSoundEffectCount = self.freeStandingSoundEffectCount.to_bytes(4, 'little')
        snakeBlockCount = self.snakeBlockCount.to_bytes(4, 'little')
        clearPipeCount = self.clearPipeCount.to_bytes(4, 'little')
        piranhaCreeperCount = self.piranhaCreeperCount.to_bytes(4, 'little')
        expandingBlockCount = self.expandingBlockCount.to_bytes(4, 'little')
        trackBlockCount = self.trackBlockCount.to_bytes(4, 'little')
        alwaysZero = self.alwaysZero.to_bytes(4, 'little')
        tileCount = self.tileCount.to_bytes(4, 'little')
        railCount = self.railCount.to_bytes(4, 'little')
        icicleCount = self.icicleCount.to_bytes(4, 'little')
        returnBytes = courseTheme + autoScrollType + screenBoundaryFlags + levelHorientation + lavaWaterHeight + lavaWaterMode + lavaWaterSpeed + minimunLavaWaterHeight  \
               + rightBoundary + topBoundary + leftBoundary + bottomBoundary + areaFlags + objectCount + freeStandingSoundEffectCount + snakeBlockCount + clearPipeCount \
               + piranhaCreeperCount + expandingBlockCount + trackBlockCount + alwaysZero + tileCount + railCount + icicleCount
        objectsBytes = fromListToBytes(self.objects, 83200)
        soundEffectsBytes = fromListToBytes(self.soundEffects, 1200)
        snakesBytes = fromListToBytes(self.snakes, 4820)
        clearPipeBytes = fromListToBytes(self.clearPipes, 58400)
        creeperBytes = fromListToBytes(self.creepers, 840)
        expandingBlockBytes = fromListToBytes(self.expandingBlocks, 440)
        trackBlockBytes = fromListToBytes(self.trackBlocks, 440)
        tilesBytes = fromListToBytes(self.tiles, 16000)
        railBytes = fromListToBytes(self.rails, 18000)
        iCicleBytes = fromListToBytes(self.icicles, 1200)
        return returnBytes +  objectsBytes + soundEffectsBytes + snakesBytes + clearPipeBytes + creeperBytes + expandingBlockBytes + trackBlockBytes + tilesBytes + railBytes + iCicleBytes
        

In [22]:
class Header(ObjectBytes):
    def __init__(self, headerBytes):
        self.startY = headerBytes[0]
        self.endY = headerBytes[1]
        self.goalXx100 = int.from_bytes(headerBytes[2:4],"little", signed = False)
        self.timeLimit = int.from_bytes(headerBytes[4:6],"little", signed = False)
        self.targetAmountClearCondition = int.from_bytes(headerBytes[6:8],"little", signed = False)
        self.year = int.from_bytes(headerBytes[8:10],"little", signed = False)
        self.month = headerBytes[10]
        self.day = headerBytes[11]
        self.hour = headerBytes[12]
        self.minute = headerBytes[13]
        self.customAutoscrollSpeed = headerBytes[14]
        self.clearConditionType = headerBytes[15]
        self.clearConditionObject = int.from_bytes(headerBytes[16:20],"little", signed = False)
        self.gameVersion = int.from_bytes(headerBytes[20:24],"little", signed = False)
        self.levelFlags = int.from_bytes(headerBytes[24:28],"little", signed = False)
        self.clearCheckTries = int.from_bytes(headerBytes[28:32],"little", signed = False)
        self.clearCheckTime = int.from_bytes(headerBytes[32:36],"little", signed = False)
        self.creationID = int.from_bytes(headerBytes[36:40],"little", signed = False)
        self.uploadID = headerBytes[40:48]
        self.gameVersionCheck = headerBytes[48:52]
        self.padding = headerBytes[52:240]
        self.unknow2 = headerBytes[240]
        self.gameStyle = headerBytes[241:244]
        self.courseName = headerBytes[244:310]#wchar16
        self.courseDescription = headerBytes[310:512]
        
    def toBytes(self):
        startY = self.startY.to_bytes(1, 'little')
        endY = self.endY.to_bytes(1, 'little')
        goalXx100 = self.goalXx100.to_bytes(2, 'little')
        timeLimit = self.timeLimit.to_bytes(2, 'little')
        targetAmountClearCondition = self.targetAmountClearCondition.to_bytes(2, 'little')
        year = self.year.to_bytes(2, 'little')
        month = self.month.to_bytes(1, 'little')
        day = self.day.to_bytes(1, 'little')
        hour = self.hour.to_bytes(1, 'little')
        minute = self.minute.to_bytes(1, 'little')   
        customAutoscrollSpeed = self.customAutoscrollSpeed.to_bytes(1, 'little') 
        clearConditionType = self.clearConditionType.to_bytes(1, 'little') 
        clearConditionObject = self.clearConditionObject.to_bytes(4, 'little') 
        gameVersion = self.gameVersion.to_bytes(4, 'little')
        levelFlags = self.levelFlags.to_bytes(4, 'little')
        clearCheckTries = self.clearCheckTries.to_bytes(4, 'little')
        clearCheckTime = self.clearCheckTime.to_bytes(4, 'little')
        creationID = self.creationID.to_bytes(4, 'little')
        unknow2 = self.unknow2.to_bytes(1, 'little')
        
        return startY + endY + goalXx100 + timeLimit + targetAmountClearCondition + year + month + day + hour + minute + customAutoscrollSpeed + clearConditionType \
               + clearConditionObject + gameVersion + levelFlags + clearCheckTries + clearCheckTime + creationID + self.uploadID \
               + self.gameVersionCheck + self.padding + unknow2 + self.gameStyle + self.courseName + self.courseDescription

In [23]:
class MarioMaker2File(ObjectBytes):
    def __init__(self, fileBytes):
        self.gameVersion = fileBytes[:4]
        self.gameVersion2 = fileBytes[4:6]
        self.padding = fileBytes[6:8]
        self.CRC32 = fileBytes[8:12]
        self.SCDL = fileBytes[12:16]
        self.header = Header(fileBytes[16:528])
        self.mainLevelArea = LevelArea(fileBytes[528:188656])
        self.subLevelArea = LevelArea(fileBytes[188656:376784])
        self.cryptoCfg = fileBytes[376784:]#non ne sono sicuro
        
    def toBytes(self):
        return self.gameVersion + self.gameVersion2 + self.padding + self.CRC32 + self.SCDL + addZeros(self.header.toBytes(), 512) + addZeros(self.mainLevelArea.toBytes(), 188128) + addZeros(self.subLevelArea.toBytes(), 188128) + self.cryptoCfg

In [24]:
levelFolder = Path("levels/")
levelsDownloaded = [f for f in listdir(levelFolder) if isfile(join(levelFolder, f))]

In [25]:
sampling = random.choices(levelsDownloaded, k=1852)

In [ ]:
levelsError = 0
for levelName in tqdm(levelsDownloaded):
    if levelName[-4:] == '.zip':
        archive = zipfile.ZipFile(join(levelFolder, levelName), 'r')
        leveldata = archive.read('course_data_000.bcd')
        archive.close()
        with open('tools/tmp/course_data_000.bcd', 'wb') as f:
            f.write(leveldata)
    else:
        archive = tarfile.open(join(levelFolder, levelName), 'r')
        archive.extract('course_data_000.bcd', path="tools/tmp/")
        archive.close()
    !.\tools\smm2dec.exe -h .\tools\tmp\course_data_000.bcd .\tools\tmp\course_data_dec.bcd >nul 2>&1
    with open('tools/tmp/course_data_dec.bcd', 'rb') as f:
        leveldata = f.read()
    testM = MarioMaker2File(leveldata)
    a = testM.toBytes()
    for idx, b in enumerate(leveldata):
        if a[idx] != b:
            levelsError += 1
            print(testM.header.courseName.decode("utf-16"))
            print(levelName)
            print(idx)
            print('--------********')
            print(a[idx])
            print(b)
            break


1-1 Paw Print Plains             
1-1_paw_print_plains_7505.zip
92740
--------********
0
10
1-1 Paw Print Plains             
1-1_paw_print_plains_8825.zip
92740
--------********
0
10
1-3 Bouncin' Up Stingby Pass ath 
1-3_bouncin_up_stingby_passath_7481.zip
90112
--------********
0
1
1-3 Bouncin' Up Stingby Pass ath 
1-3_bouncin_up_stingby_passath_8853.zip
90112
--------********
0
1
1111                             
1111_4484.zip
149104
--------********
0
1
111                              
111_3513.zip
149104
--------********
0
1
11vv                             
11vv_4621.zip
149720
--------********
0
1
1234impossible                   
1234impossible_12850.zip
149720
--------********
0
1
123                              
123_3470.zip
149720
--------********
0
1
1                                
1_10270.zip
149104
--------********
0
1
1                                
1_1154.zip
149720
--------********
0
1
1                                
1_12705.zip
149720
--------********
0
1
1   

宇宙のどこにも見当たらないような　約束の口づけをマリメでしよう  
5dc42c4900678f1b00ceb70a.tar
90112
--------********
0
1
爽快！ドンケツ退治！　　　                    
5df80f6f00eb74d600e3338a.tar
337892
--------********
0
1
Pit of PePanga: POW-a-Panga      
5df80f9200b0a4e500e333bb.tar
337496
--------********
0
1
Casino Royale                    
5df80fc0008d773200e33426.tar
93324
--------********
0
12
[4P] Versus: Hot Potato! v1.7    
5df80fc20035fb9600e3344d.tar
279116
--------********
0
4
6-1: The Bully Of The Beest ion  
5df8103d0022918500e334a9.tar
92740
--------********
0
10
[SDW2] 5-2 Slippery Spiky Subway 
5df811a5008dc63a00e33516.tar
276984
--------********
0
4
Quinn 4                          
5ec4c48700e312ca00bff347.tar
90404
--------********
0
2
★超爽快★春の爽快SPECIAL 6　☆いつもより爽快コース☆  
5ec4c488008de0cd00bff355.tar
278532
--------********
0
2
超・超・爽快♪ 最高にHappy！ ★爽快メドレー☆  game 
5ec4c48800df28f300bff34b.tar
337364
--------********
0
1
超爽快♪　ストレス発散！　爽快メドレー♪#4           
5ec4c497008800f800bff35e.tar
337408
--------********
0
1
超

Nivel en creación                
5ed8d00200bf734600e7ed82.tar
278240
--------********
0
1
Hell pipes        s              
5ed8d04700ed14ae00e7ed90.tar
85964
--------********
0
1
Larl's 3D trollventure           
5ed8d094005a4a6000e7eda1.tar
149544
--------********
0
1
Sabloid                          
5ed8d0c9003b687e00e7edaf.tar
90112
--------********
0
1
pick the way to pom pom          
5ed8d0eb002ae99600e7edb8.tar
90404
--------********
0
2
Mansion of Misdirection          
5ed8d1650044d23700e7edd6.tar
281160
--------********
0
11
One Screen Speedrun: Mean Thwomp 
5ed8e21c0024c1fc00e7eebd.tar
149148
--------********
0
1
Chaotic Course / カオスステージ ♪～★     
5ed8eb4600e8099b00e7f00d.tar
93324
--------********
0
12
Zelda: Fire Temple ple v2        
5ed8ed1900c82db200e7f051.tar
337364
--------********
0
1
[SDW2] 5-2 Slippery Spiky Subway 
5ed8ee0100f6dafe00e7f07a.tar
276984
--------********
0
4
Trials of the Four Chambers      
5ed8f509004f304000e7f16f.tar
337452
--------********
0
1
R

In [ ]:
levelsError

In [ ]:
archive = zipfile.ZipFile(join(levelFolder, '007_-_bond_mario_bond__2958.zip'), 'r')
leveldata = archive.read('course_data_000.bcd')
archive.close()

In [ ]:
len(leveldata)

In [ ]:
archive = tarfile.open(join(levelFolder, '5ed941a20050e57600223491.tar'), 'r')
archive.extract('course_data_000.bcd', path="tools/tmp/")
archive.close()

In [ ]:
!.\tools\smm2dec.exe -h .\tools\tmp\course_data_000.bcd .\tools\tmp\course_data_dec.bcd

In [ ]:
with open('tools/tmp/course_data_dec.bcd', 'rb') as f:
    leveldata = f.read()

In [ ]:
leveldata[149327]

In [ ]:
testM = MarioMaker2File(leveldata)
len(testM.toBytes())

In [ ]:
testM.mainLevelArea.expandingBlockCount

In [ ]:
testM.header.gameStyle.decode("utf-8")

In [ ]:
testM.header.courseName.decode("utf-16")

In [ ]:
a = testM.toBytes()
print(len(a))
for idx, b in enumerate(leveldata):
    if a[idx] != b:
        print('porco dio')
        print(idx)
        break

In [ ]:
CryptoCfg

In [ ]:
courseName.decode("utf-16")

In [ ]:
courseDescription.decode("utf-16")

In [ ]:
gameStyle.decode("utf-8")